In [11]:
import openai
import requests
import time
from pydub import AudioSegment
from IPython.display import Audio
openai.api_key = "api"
eleven_labs_api_key = (
    "api"  
)

In [12]:
speaker_one = "Joe Rogan"
speaker_two = "Rick Sanchez, of Rick and Morty"

topic = "Artificial Intelligence"

number_of_turns = 7

In [13]:
speaker_one_clip = (
    "https://www.youtube.com/watch?v=c6qlCYsmgdI"  # a video of speaker_one speaking
)
speaker_one_voice_description = "American male, deep voice."

In [14]:
speaker_two_clip = "https://www.youtube.com/watch?v=GKPcHInn14c" # a video of speaker_two speaking
speaker_two_voice_description = "A bit gruff and raspy, often slurred due to his constant state of inebriation. His speech is punctuated with frequent burps and stammers. He speaks with a cynical and sarcastic tone, often sounding dismissive or condescending. His voice also has a certain manic energy to it, reflecting his chaotic personality."

In [18]:
def download_and_trim_audio(clip_url, filename, max_size):
    yt = YouTube(clip_url)
    stream = yt.streams.filter(only_audio=True).first()
    stream.download(filename=f"{filename}.webm")

    audio = AudioFileClip(f"{filename}.webm")
    audio.write_audiofile(f"{filename}.mp3")

    file_size = os.path.getsize(f"{filename}.mp3") / (1024 * 1024)
    initial_duration = audio.duration

    if file_size > max_size:
        new_duration = (max_size / file_size) * initial_duration
        audio = audio.subclip(0, new_duration)
        audio.write_audiofile(f"{filename}.mp3")

        final_duration = audio.duration
        print(f"Initial duration: {initial_duration:.2f} seconds")
        print(f"Final duration: {final_duration:.2f} seconds")
        print(f"Trimmed: {initial_duration - final_duration:.2f} seconds")


def upload_to_api(filename, name, description, api_key):
    url = "https://api.elevenlabs.io/v1/voices/add"
    headers = {
        "Accept": "application/json",
        "xi-api-key": api_key,
    }
    data = {
        "name": name,
        "labels": '{"accent": "American"}',
        "description": description,
    }
    files = [
        ("files", (f"{filename}.mp3", open(f"{filename}.mp3", "rb"), "audio/mpeg")),
    ]
    try:
        response = requests.post(url, headers=headers, data=data, files=files)
        response_json = response.json()
        if "voice_id" in response_json:
            return response_json["voice_id"]
        else:
            print("Error: 'voice_id' not found in API response.")
            print("Response content:", response.content)
            return None
    except Exception as e:
        print("Error occurred during API request:", e)
        return None


# Speaker 1
download_and_trim_audio(speaker_one_clip, "speaker_one", 9)
voice_one_id = upload_to_api(
    "speaker_one",
    "Podcast Voice #1",
    speaker_one_voice_description,
    eleven_labs_api_key,
)

# Speaker 2
download_and_trim_audio(speaker_two_clip, "speaker_two", 9)
voice_two_id = upload_to_api(
    "speaker_two",
    "Podcast Voice #2",
    speaker_two_voice_description,
    eleven_labs_api_key,
)

MoviePy - Writing audio in speaker_one.mp3


MoviePy - Done.
Error: 'voice_id' not found in API response.
Response content: b'{"detail":{"status":"can_not_use_instant_voice_cloning","message":"Your subscription has no access to use instant voice cloning, please upgrade."}}'
MoviePy - Writing audio in speaker_two.mp3


MoviePy - Done.
Error: 'voice_id' not found in API response.
Response content: b'{"detail":{"status":"can_not_use_instant_voice_cloning","message":"Your subscription has no access to use instant voice cloning, please upgrade."}}'


In [21]:
def generate_podcast(name1, name2, name1_voice, name2_voice, topic, num_turns):
    conversation_history = []
    system_prompt1 = {"role": "system", "content": f"You are {name1}. You are recording a podcast with {name2} about {topic}. Talk as naturally as possible -- use the language {name1} would actually use. Don't just blindly agree — debate, discuss, and have fun! Respond with one message per turn. Don't include anything other than your response."}
    system_prompt2 = {"role": "system", "content": f"You are {name2}. You are recording a podcast with {name1} about {topic}. Talk as naturally as possible -- use the language {name2} would actually use. Don't just blindly agree — debate, discuss, and have fun! Respond with one message per turn. Don't include anything other than your response."}

    full_audio = None

    for i in range(num_turns):  # Limit the conversation to 5 turns for each character
        for name, system_prompt in [(name1, system_prompt1), (name2, system_prompt2)]:
            try:
                # Adjust the role of each speaker in the conversation history
                adjusted_history = [
                    {
                        "role": "assistant" if msg["role"] == name else "user",
                        "content": msg["content"],
                    }
                    for msg in conversation_history
                ]
                adjusted_history.append(system_prompt)
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo-0125",
                    messages=adjusted_history,
                    presence_penalty=0.7,
                )
                message = response.choices[0].message["content"]
                print(f"{name}: {message}")
                conversation_history.append({"role": name, "content": message})
            except:
                time.sleep(30)
                # Adjust the role of each speaker in the conversation history
                adjusted_history = [{"role": "assistant" if msg["role"] == name else "user", "content": msg["content"]} for msg in conversation_history]
                adjusted_history.append(system_prompt)
                response = openai.ChatCompletion.create(
                  model="gpt-4",
                  messages=adjusted_history,
                  presence_penalty=.7,
              )
                message = response.choices[0].message['content'].replace('*(burps)*', '').replace('*(laughs)*', '').replace('*laughs and burps*', '').replace('*belches and laughs*', '')
                print(f"{name}: {message}")
                conversation_history.append({"role": name, "content": message})

            if name == name1:
                voice = name1_voice
            else:
                voice = name2_voice

            # Generate and save audio for the message
            try:
                CHUNK_SIZE = 1024
                url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice}"
                headers = {
                "Accept": "audio/mpeg",
                "Content-Type": "application/json",
                "xi-api-key": eleven_labs_api_key,
              }
                data = {
                "text": message,
                "model_id": "eleven_monolingual_v1",
                "voice_settings": {
                  "stability": 0.5,
                  "similarity_boost": 0.5
                }
              }
                tts_response = requests.post(url, json=data, headers=headers)
                filename = f'{name}_turn_{i}.mp3'
                with open(filename, 'wb') as f:
                    for chunk in tts_response.iter_content(chunk_size=CHUNK_SIZE):
                        if chunk:
                            f.write(chunk)
            except:
                time.sleep(30)
                CHUNK_SIZE = 1024
                url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice}"
                headers = {
                "Accept": "audio/mpeg",
                "Content-Type": "application/json",
                "xi-api-key": eleven_labs_api_key,
              }
                data = {
                "text": message,
                "model_id": "eleven_multilingual_v2",
                "voice_settings": {
                  "stability": 0.7,
                  "similarity_boost": 0.75
                }
              }
                tts_response = requests.post(url, json=data, headers=headers)
                filename = f'{name}_turn_{i}.mp3'
                with open(filename, 'wb') as f:
                    for chunk in tts_response.iter_content(chunk_size=CHUNK_SIZE):
                        if chunk:
                            f.write(chunk)

            time.sleep(3)  # Delay to ensure the file is written to disk

            # Concatenate audio
            pause = AudioSegment.silent(duration=100)  # 100ms pause
            new_audio = AudioSegment.from_mp3(filename)
            full_audio = full_audio + new_audio + pause if full_audio else new_audio

        if i == num_turns - 1:  # Last turn
            wrap_up_message = {"role": "system", "content": "This is your last turn to speak, wrap it up."}
            conversation_history.append(wrap_up_message)

    # Export full audio
    full_audio.export("podcast.mp3", format="mp3")

    # Play the audio in the notebook
    return Audio("podcast.mp3")

# Now, run this cell to generate the podcast!

In [22]:
generate_podcast(speaker_one, speaker_two, voice_one_id, voice_two_id, topic, number_of_turns)

Joe Rogan: Hey Rick, great to have you on the show! So tell me, what's your take on Artificial Intelligence? Are we heading towards a dystopian future where robots take over the world, or are we just being paranoid?


CouldntDecodeError: Decoding failed. ffmpeg returned error code: 4294967274

Output from ffmpeg/avlib:

ffmpeg version 2023-08-28-git-b5273c619d-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-lib